# Train a deep CNN on XPS data on Google Colab

In this notebook, we will train a deep convolutional network on iron XPS spectra made up of single iron reference spectra.

## Setup

### Mount google drive, change working directory

In [ ]:
# Mount drive
from google.colab import drive
import os

drive.mount('/content/drive')

# Change working path
os.chdir('/content/drive/My Drive/app')

### Install packages and import modules

In [ ]:
# Install packages
!pip install python-docx

# Import standard modules and magic commands
import datetime
import numpy as np
import pytz
import importlib

# Magic commands
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Disable tf warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf

## Classification

### Load custom modules

In [ ]:
try:
    import importlib
    importlib.reload(classifier)
    importlib.reload(clfutils)
    print('Modules were reloaded.')
except:
    import xpsdeeplearning.network.classifier as classifier
    import xpsdeeplearning.network.utils as clfutils
    print('Modules were loaded.')

### Setting up the parameters & folder structure

In [ ]:
np.random.seed(502)
time = '20200715_15h18m' # datetime.datetime.now().astimezone(pytz.timezone('Europe/Berlin')).strftime("%Y%m%d_%Hh%Mm")
data_name = 'Fe_single_4_classes'

label_values = ['Fe metal', 'FeO', 'Fe3O4', 'Fe2O3']

clf = classifier.ClassifierSingle(time = time,
                                  data_name = data_name,
                                  labels = label_values)

### Load and inspect the data

In [ ]:
input_filepath = r'/content/drive/My Drive/app/datasets/20200605_iron_single_500000.h5'
train_test_split = 0.2
train_val_split = 0.2
no_of_examples = 100#000

X_train, X_val, X_test, y_train, y_val, y_test,\
    aug_values_train, aug_values_val, aug_values_test =\
        clf.load_data_preprocess(input_filepath = input_filepath,
                                 no_of_examples = no_of_examples,
                                 train_test_split = train_test_split,
                                 train_val_split = train_val_split, 
                                 augmentation_values = True)
        
# Check how the examples are distributed across the classes.
class_distribution = clf.check_class_distribution()
clf.plot_class_distribution()
clf.plot_random(no_of_spectra = 10, dataset = 'train')  

### Design the model

In [ ]:
try:
    importlib.reload(models)
    print('Models module was reloaded.')
except:
    import xpsdeeplearning.network.models as models
    print('Models module was loaded.')

In [ ]:
from tensorflow.keras.layers import Input, Dense, Flatten, concatenate
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import AveragePooling1D, MaxPooling1D
#from tensorflow.keras.layers import BatchNormalization

class CustomCNN(models.EmptyModel):
    def __init__(self, inputshape, num_classes):      
        input_1 = Input(shape = inputshape)
                
        conv_1_short = Conv1D(4, 5, padding = 'same',
                            activation = 'relu')(input_1)
        conv_1_medium = Conv1D(4, 10, padding = 'same',
                             activation = 'relu')(input_1)
        conv_1_long = Conv1D(4, 15, padding = 'same',
                           activation = 'relu')(input_1)
        sublayers = [conv_1_short, conv_1_medium, conv_1_long]
        merged_sublayers = concatenate(sublayers)
        
        conv_2 = Conv1D(4, 5, activation='relu')(merged_sublayers)
        average_pool_1 = AveragePooling1D()(conv_2)
        
        flatten_1 = Flatten()(average_pool_1)
        drop_1 = Dropout(0.2)(flatten_1)
        dense_1 = Dense(1000, activation = 'relu')(drop_1)
        
        dense_2 = Dense(num_classes, activation = 'softmax')(dense_1)
        
        no_of_inputs = len(sublayers)

        super(CustomCNN, self).__init__(inputs = input_1,
                                        outputs = dense_2,
                                        inputshape = inputshape,
                                        num_classes = num_classes,
                                        no_of_inputs = no_of_inputs, 
                                        name = 'Custom_CNN')

### Build the model

In [ ]:
clf.model = CustomCNN(clf.input_shape, clf.num_classes)

# Alternative: Build model from available models
#clf.model = models.CustomCNN(clf.input_shape, clf.num_classes)

### Compile and summarize the model

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

learning_rate = 1e-05
optimizer = Adam(learning_rate = learning_rate) 
loss = CategoricalCrossentropy()

# Alternative: Compile model with build-in loss function
clf.model.compile(loss = loss,
                  optimizer = optimizer, 
                  metrics = ['accuracy'])

# Plot summary and save model plot.
clf.summary()
clf.save_and_print_model_image()

### Train

In [ ]:
epochs = 2#150
batch_size = 32

hist = clf.train(checkpoint = True,
                 early_stopping = False,
                 tb_log = True, 
                 csv_log = True,
                 epochs = epochs, 
                 batch_size = batch_size,
                 verbose = 1)

### Generate loss and accuracy graphs

In [ ]:
dir_name = clf.time + '_' + clf.data_name
graph = clfutils.TrainingGraphs(clf.history, dir_name)
graph.plot_loss()
graph.plot_accuracy()

### Evaluate on test data

In [ ]:
score = clf.evaluate()
test_loss, test_accuracy = score[0], score[1]
print('Test loss: ' + str(np.round(test_loss, decimals=3)))
print('Test accuracy: ' + str(np.round(test_accuracy, decimals=3)))

###  Prediction on train & test data

In [ ]:
pred_train, pred_test = clf.predict()
pred_train_classes, pred_test_classes = clf.predict_classes()

### Show some predictions

#### 10 random training samples

In [ ]:
clf.plot_random(no_of_spectra = 10, dataset = 'train', with_prediction = True)  

#### 10 random test samples

In [ ]:
clf.plot_random(no_of_spectra = 6, dataset = 'test', with_prediction = True)  

### Show wrong predictions

In [ ]:
clf.show_wrong_classification()

In [ ]:
def show_wrong_classification():
    import matplotlib.pyplot as plt
    binding_energy = np.arange(694, 750.05, 0.05)
        
    wrong_pred_args = [1,2,3,4,5,6]
        
    for i in range(clf.pred_test.shape[0]): 
        argmax_class_true = np.argmax(clf.y_test[i,:], axis = 0)
        argmax_class_pred = np.argmax(clf.pred_test[i,:], axis = 0)
            
    if argmax_class_true != argmax_class_pred:
        wrong_pred_args.append(i)
    no_of_wrong_pred = len(wrong_pred_args)
    print('No. of wrong predictions on the test data: ' +\
          str(no_of_wrong_pred))
        
    if no_of_wrong_pred > 0:
        no_of_cols = 5
        no_of_rows = int(no_of_wrong_pred/no_of_cols)
        if (no_of_wrong_pred % no_of_cols) != 0:
            no_of_rows += 1

        fig, axs = plt.subplots(nrows = no_of_rows, ncols = no_of_cols)
        plt.subplots_adjust(left = 0.125, bottom = 0.5,
                            right=4.8, top = no_of_rows,
                            wspace = 0.2, hspace = 0.2)

        for n in range(no_of_wrong_pred):
            arg = wrong_pred_args[n]
            intensity = clf.X_test[arg]
            
            real_y = ('Real: ' + \
                str(clf.y_test[arg]) + '\n')
            aug = clf._write_aug_text(dataset = 'test', index = arg)
            # Round prediction and sum to 1
            tmp_array = np.around(clf.pred_test[arg], decimals = 4)
            row_sums = tmp_array.sum()
            tmp_array = tmp_array / row_sums
            tmp_array = np.around(tmp_array, decimals = 2)
            pred_y = ('Prediction: ' +\
                      str(tmp_array) + '\n')
            pred_label = ('Predicted label: ' +\
                          str(clf.pred_test_classes[arg,0]))
            labels = clf.y_test[arg]
            for j, value in enumerate(labels):
                if value == 1:
                    label = str(clf.label_values[j])
                    label =  ('Real label: ' + label + '\n')
                
            text = real_y + pred_y + label + pred_label + aug
            
            row, col = int(n/no_of_cols), n % no_of_cols
            axs[row, col].plot(np.flip(binding_energy),intensity)
            axs[row, col].invert_xaxis()
            axs[row, col].set_xlim(750.05,694)
            axs[row, col].set_xlabel('Binding energy (eV)')
            axs[row, col].set_ylabel('Intensity (arb. units)')  
            axs[row, col].text(0.025, 0.35, text,
                                horizontalalignment='left',
                                verticalalignment='top',
                                transform = axs[row, col].transAxes,
                                fontsize = 12)
show_wrong_classification()                

### Save model and data

In [ ]:
clf.save_model()
clf.save_hyperparams()
clf.shelve_results(full = False)

### Generate report

In [ ]:
dir_name = clf.time + '_' + clf.data_name
rep = clfutils.Report(dir_name)  
rep.write()   

## Continue training

In [ ]:
# Reload and train for more epochs
# model_path = r'C:\Users\pielsticker\Lukas\MPI-CEC\Projects\xpsdeeplearning\saved_models\20200608_17h51m_Fe_single_4_classes_CNN_simple' 
clf.load_model()
clf.model.compile(loss = loss,
                  optimizer = optimizer, 
                  metrics = ['accuracy'])

new_learning_rate = 1e-04
epochs = 2
batch_size = 32
hist = clf.train(checkpoint = True,
                 early_stopping = False,
                 tb_log = True, 
                 csv_log = True,
                 epochs = epochs,
                 batch_size = batch_size, 
                 new_learning_rate = new_learning_rate) # Learning rate can be changed for retraining

from tensorflow.keras import backend as K
print('New learning rate: ' +\
      str(K.eval(clf.model.optimizer.lr)))

dir_name = clf.time + '_' + clf.data_name
graphs = clfutils.TrainingGraphs(hist, dir_name)
graph.plot_loss()
graph.plot_accuracy()

score = clf.evaluate()
test_loss, test_accuracy = score[0], score[1]

pred_train, pred_test = clf.predict()
pred_train_classes, pred_test_classes = clf.predict_classes()
clf.plot_random(no_of_spectra = 6, dataset = 'train', with_prediction = True)  
clf.plot_random(no_of_spectra = 6, dataset = 'test', with_prediction = True)  

clf.save_model()
clf.save_hyperparams()
clf.shelve_results(full = False)  

rep = clfutils.Report(dir_name)  
rep.write()  

### Save output of notebook

In [ ]:
from IPython.display import Javascript, display
from nbconvert import HTMLExporter

def save_notebook():
    display(Javascript("IPython.notebook.save_notebook()"),
            include=['application/javascript'])

def output_HTML(read_file, output_file):
    import codecs
    import nbformat
    exporter = HTMLExporter()
    # read_file is '.ipynb', output_file is '.html'
    output_notebook = nbformat.read(read_file, as_version=4)
    output, resources = exporter.from_notebook_node(output_notebook)
    codecs.open(output_file, 'w', encoding='utf-8').write(output)

import time
import os

time.sleep(20)
save_notebook()
print('Notebook saved!')
time.sleep(30)
current_file = '/content/drive/My Drive/app/xpsdeeplearning/train_colab_single.ipynb'
output_file = os.path.join(clf.log_dir,'train_colab_single_out.html')
output_HTML(current_file, output_file)